# Title: SIFT Missense Predictions for Genomes and 1000 Genomes Data
# Course: MED263, "Bioinformatics Applications To Human Disease"
# Preparer: Nathaniel Delos Santos

# 1) Introduction
In this practical, you are going to use linux command line tools, the SIFT4G Variant annotator, and data from the 1000 Genomes Project to predict deleterious missense mutations from human samples. Predicting variant deleteriousness is an important part of analyzing human genome variants in disease, because it provides insight into which genes have been affected by a variant, and how bad the effect might be.  
After this tutorial, you should be able to:  
 - Download variant information from the Ensembl project
 - Download aligned sequence data from the 1000 Genomes project
 - Call variants from aligned sequence data
 - Annotate variants from aligned sequence data with deleteriousness and amino acid change predictions
 - Prioritize variants by a deleteriousness score
 - Perform data manipulation with basic command line tools in BASH

## 1.1) Download SIFT 4G
SIFT 4G, (Sorting Intolerant From Tolerant, For Genomes) uses variant calls to predict what amino acid substitutions occur, and how deleterious they are. SIFT 4G requires Java and a reference database to run. 

We will download SIFT 4G directly from their website at (http://sift.bii.a-star.edu.sg/sift4g/) using wget. Make sure wget is installed on your system.

In [1]:
%%bash
wget -q http://sift.bii.a-star.edu.sg/sift4g/SIFT4G_Annotator_v2.4.jar

## 1.2) Download Homo Sapiens Database (GRCh38.78) for SIFTG
We must download the reference database for GRCh38, the newest version of the human genome reference available from Ensembl. We will download SIFT 4G's version of this database directly from their website. Make sure to choose GRCh38.78. Decompress the 

In [2]:
%%bash
wget -q http://sift.bii.a-star.edu.sg/sift4g/public/Homo_sapiens/GRCh38.78.zip -O GRCh38.78.zip

In [3]:
%%bash
unzip GRCh38.78.zip

Archive:  GRCh38.78.zip
   creating: GRCh38.78/
  inflating: GRCh38.78/CHR_HSCHR19KIR_FH15_A_HAP_CTG3_1_SIFTDB_stats.txt  
  inflating: GRCh38.78/CHR_HSCHR4_6_CTG12_SIFTDB_stats.txt  
 extracting: GRCh38.78/KI270750.1.gz  
 extracting: GRCh38.78/KI270743.1.regions  
  inflating: GRCh38.78/CHR_HSCHR2_4_CTG1.regions  
  inflating: GRCh38.78/CHR_HSCHR1_1_CTG31.regions  
  inflating: GRCh38.78/CHR_HSCHR19KIR_RP5_B_HAP_CTG3_1.regions  
  inflating: GRCh38.78/CHR_HSCHR17_4_CTG4_SIFTDB_stats.txt  
  inflating: GRCh38.78/#!genome-version  
  inflating: GRCh38.78/CHR_HSCHR22_1_CTG6.gz  
  inflating: GRCh38.78/CHR_HSCHR19KIR_FH06_A_HAP_CTG3_1_SIFTDB_stats.txt  
  inflating: GRCh38.78/CHR_HSCHR9_1_CTG5.regions  
  inflating: GRCh38.78/11.gz         
 extracting: GRCh38.78/KI270744.1.gz  
  inflating: GRCh38.78/CHR_HSCHR6_MHC_SSTO_CTG1.regions  
  inflating: GRCh38.78/CHR_HSCHR6_1_CTG8_SIFTDB_stats.txt  
  inflating: GRCh38.78/CHR_HSCHR17_2_CTG5.gz  
  inflating: GRCh38.78/16.gz         
  inflati

## 1.3) SAMTools
SAMTools is a general toolkit for use with aligned sequencing data. We will use it here to call variants from sequence alignments, using the 'samtools mpileup' command. We will install version 1.4 here, since the specific version matters for our purposes.
Make sure that GCC and your build environment are up to date.

In [4]:
%%bash
wget -q https://github.com/samtools/samtools/releases/download/1.4/samtools-1.4.tar.bz2 -O samtools-1.4.tar.bz2

In [5]:
%%bash
tar -vxjf samtools-1.4.tar.bz2
cd samtools-1.4
./configure
make
cd ..

samtools-1.4/
samtools-1.4/AUTHORS
samtools-1.4/ChangeLog.old
samtools-1.4/INSTALL
samtools-1.4/LICENSE
samtools-1.4/Makefile
samtools-1.4/Makefile.mingw
samtools-1.4/NEWS
samtools-1.4/README
samtools-1.4/bam.c
samtools-1.4/bam.h
samtools-1.4/bam2bcf.c
samtools-1.4/bam2bcf.h
samtools-1.4/bam2bcf_indel.c
samtools-1.4/bam2depth.c
samtools-1.4/bam_addrprg.c
samtools-1.4/bam_aux.c
samtools-1.4/bam_cat.c
samtools-1.4/bam_color.c
samtools-1.4/bam_endian.h
samtools-1.4/bam_flags.c
samtools-1.4/bam_import.c
samtools-1.4/bam_index.c
samtools-1.4/bam_lpileup.c
samtools-1.4/bam_lpileup.h
samtools-1.4/bam_mate.c
samtools-1.4/bam_md.c
samtools-1.4/bam_plbuf.c
samtools-1.4/bam_plbuf.h
samtools-1.4/bam_plcmd.c
samtools-1.4/bam_quickcheck.c
samtools-1.4/bam_reheader.c
samtools-1.4/bam_rmdup.c
samtools-1.4/bam_rmdupse.c
samtools-1.4/bam_sort.c
samtools-1.4/bam_split.c
samtools-1.4/bam_stat.c
samtools-1.4/bam_tview.c
samtools-1.4/bam_tview.h
samtools-1.4/bam_tview_curses.c
samtools-1.4/bam_tview_html.c


config.mk:45: htslib-1.4/htslib_static.mk: No such file or directory
ar: `u' modifier ignored since `D' is the default (see `U')


## 1.4) BCFTools
BCFTools is a general toolkit for use with variant call format (VCF) files. We will use it here to filter and query variants. We install version 1.4 here as we did for SAMTools

In [6]:
%%bash
wget -q https://github.com/samtools/bcftools/releases/download/1.4/bcftools-1.4.tar.bz2 -O bcftools-1.4.tar.bz2

In [7]:
%%bash
tar -vxjf bcftools-1.4.tar.bz2
cd bcftools-1.4
./configure
make
cd ..

bcftools-1.4/
bcftools-1.4/AUTHORS
bcftools-1.4/HMM.c
bcftools-1.4/HMM.h
bcftools-1.4/INSTALL
bcftools-1.4/LICENSE
bcftools-1.4/Makefile
bcftools-1.4/NEWS
bcftools-1.4/README
bcftools-1.4/bam2bcf.c
bcftools-1.4/bam2bcf.h
bcftools-1.4/bam2bcf_indel.c
bcftools-1.4/bam_sample.c
bcftools-1.4/bam_sample.h
bcftools-1.4/bcftools.h
bcftools-1.4/bin.c
bcftools-1.4/bin.h
bcftools-1.4/call.h
bcftools-1.4/ccall.c
bcftools-1.4/consensus.c
bcftools-1.4/convert.c
bcftools-1.4/convert.h
bcftools-1.4/csq.c
bcftools-1.4/doc/
bcftools-1.4/doc/bcftools.1
bcftools-1.4/doc/bcftools.txt
bcftools-1.4/doc/docbook-xsl.css
bcftools-1.4/em.c
bcftools-1.4/filter.c
bcftools-1.4/filter.h
bcftools-1.4/gvcf.c
bcftools-1.4/gvcf.h
bcftools-1.4/hclust.c
bcftools-1.4/hclust.h
bcftools-1.4/khash_str2str.h
bcftools-1.4/kheap.h
bcftools-1.4/kmin.c
bcftools-1.4/kmin.h
bcftools-1.4/main.c
bcftools-1.4/mcall.c
bcftools-1.4/mpileup.c
bcftools-1.4/mw.h
bcftools-1.4/peakfit.c
bcftools-1.4/peakfit.h
bcftools-1.4/ploidy.c
bcftools-1

bash: line 3: ./configure: No such file or directory
Makefile:34: htslib-1.4/htslib_static.mk: No such file or directory


# 2) Data

## 2.1) Craig Venter Germline Variations
Craig Venter's genome was among the first sequenced. These Variant Call Format (VCF) files summarize the variants observed in his genome from the GRCh38.78 reference.

In [8]:
%%bash
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Venter.vcf.gz -O Venter.vcf.gz
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Venter.vcf.gz.tbi -O Venter.vcf.gz.tbi

## Question 1) 
How many variants are in the Venter VCF?

## Answer 1)

In [9]:
%%bash
zcat Venter.vcf.gz|grep -v '#'|wc -l

3266109


3266109 Variants

## 2.2) James Watson Germline Variations
James Watson is famous for discovering the double helix structure of DNA with Francis Crick. He has his own tribute in VCF format here.

In [10]:
%%bash
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Watson.vcf.gz -O Watson.vcf.gz
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Watson.vcf.gz.tbi -O Watson.vcf.gz.tbi

## 2.3) 1000 Genomes human sample exome data
The 1000 Genomes project was an international effort to catalog most variants with more than 1% frequency in the human population. It is a valuable source of human sequencing data. We will not be using the VCFs directly, but instead will be analyzing aligned sequences from a single human sample.

### 2.3.1) CRAM files
CRAM files are compressed sequence alignment files that use delta compression from a reference to store sequence information, rather than containining the sequence data themselves. Therefore, we must download the CRAM file, CRAM index, and the corresponding reference files to use them.

In [11]:
%%bash
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/CEU/NA06984/exome_alignment/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram -O NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/CEU/NA06984/exome_alignment/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram.crai -O NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram.crai

The reference files for the CRAM file are downloaded below

In [12]:
%%bash
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa -O GRCh38_full_analysis_set_plus_decoy_hla.fa
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa.fai -O GRCh38_full_analysis_set_plus_decoy_hla.fa.fai

## Question 2)
From the README provided by the 1000 Genomes Project (ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/README.1000genomes.GRCh38DH.alignment) , what steps have already been performed for these CRAM files to make them ready for analysis?

## Answer 2)
1. Read alignment
2. Local realignment around Indels
3. Recalibration of base quality scores
4. Marking of duplicate reads
5. Merging multiple sequencing libraries into a single sample alignment file
6. Lossless compression using CRAM

# 3) Methods/Results
We will now run SIFT4G to predict the deleteriousness of variants found in the Venter VCF.

## 3.1) Analysis of Craig Venter germline variants
First we must decompress the gzipped VCF to an uncompressed VCF using zcat.

SIFT4G is run using java, so we must call it using 'java -jar', passing the SIFT4G program as the '-jar' option. The '-c' option will run SIFT4G in command line mode, and the '-t' option will cause SIFT4G to output additional annotations for each transcript of a gene affected. The '-i' option specifies the input VCF, in this case 'Venter.vcf'. The '-d' option specifies the database we will be using, in this case the GRCh38.78 database. The '-r' option will determine where the results of the SIFT annotation will be located relative to our current directory.

In [13]:
%%bash
zcat Venter.vcf.gz > Venter.vcf
java -jar SIFT4G_Annotator_v2.4.jar -c -t -i Venter.vcf -d GRCh38.78  -r Venter.SIFT4G

Start Time for SIFT4G code: Mon Mar 27 01:34:17 PDT 2017
Updates:
No updates from server!! Please go to http:sift-dna.org for updates.

Started Running .......
Running in Multitranscripts mode

Chromosome	WithSIFT4GAnnotations	WithoutSIFT4GAnnotations	Progress
MT			1			0			Completed : 1/25
Y			130			18578			Completed : 2/25
22			2914			22450			Completed : 3/25
20			5175			66630			Completed : 4/25
13			9175			116297			Completed : 5/25
21			7318			59106			Completed : 6/25
X			3941			79652			Completed : 7/25
10			10038			163272			Completed : 8/25
9			12656			155721			Completed : 9/25
19			9448			57848			Completed : 10/25
18			11794			79902			Completed : 11/25
7			10687			138041			Completed : 12/25
17			8831			79089			Completed : 13/25
16			13997			99378			Completed : 14/25
14			12880			86113			Completed : 15/25
6			9155			147054			Completed : 16/25
11			15317			153409			Completed : 17/25
15			16910			83760			Completed : 18/25
12			17570			132459			Completed : 19/25
8			18309			155222			Co

## Question 3)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 3)
8831 annotated, 79089 unnannotated

### 3.1.1 SIFT 4G Output
The output of SIFT 4G includes a VCF file and an excel (.xls) file that describe the amino acid changes and the predicted deleteriousness of each variant. The excel file is formatted similarly to a tab-separated values file, with the exception of a carriage return ('\r') before each new line. We will use this to navigate the SIFT 4G output.

## Question 4)
How many columns Does the SIFT4G output contain? What does each column contain?

## Answer 4)

In [14]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|head -n1|tr '\t' '\n'|cat -n

     1	CHROM
     2	POS
     3	REF_ALLELE
     4	ALT_ALLELE
     5	TRANSCRIPT_ID
     6	GENE_ID
     7	GENE_NAME
     8	REGION
     9	VARIANT_TYPE
    10	REF_AMINO
    11	ALT_AMINO
    12	AMINO_POS
    13	SIFT_SCORE
    14	SIFT_MEDIAN
    15	NUM_SEQS
    16	dbSNP
    17	SIFT_PREDICTION


17 Columns, contents are described above.

## Question 5)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 5)

In [15]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1561


1561 deleterious variants.

## Question 6)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 6)

In [16]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> Venter.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter.SIFT4G.genes_with_deleterious_variants.txt
cat Venter.SIFT4G.genes_with_deleterious_variants.txt

1186 Venter.SIFT4G.genes_with_deleterious_variants.txt
A2ML1
ABCA10
ABCA6
ABCA7
ABCC8
ABCD1
AC008686.1
AC073657.1
ACACB
ACADS
ACAN
ACSM2A
ACTL9
ACTN3
ACTRT2
ADAM19
ADAM7
ADAMDEC1
ADAMTS13
ADAMTS14
ADAMTSL3
ADCK5
ADH1C
ADH4
ADORA3
ADPGK
AGAP10
AGT
AHCY
AHNAK
AIM1L
AKAP13
AKR1C2
ALDH1B1
ALDH5A1
ALOX15
ALPK2
ALPK3
ALPP
ALX4
AMACR
AMPD3
AMY2A
ANAPC1
ANKDD1A
ANKLE1
ANKRD1
ANKRD30A
ANKRD33
ANKRD35
ANKRD36
ANKRD36B
ANKRD36C
ANKRD60
ANKRD62
ANXA13
AOAH
AP1G2
AP2S1
APIP
APOA1BP
APOB
APOBEC3H
APOC4
APOC4-APOC2
APOL4
AQP12A
AQP7
ARHGAP9
ARHGEF19
ARHGEF3
ARHGEF37
ARL5C
ARMC4
ARMC9
ARPP21
ARR3
ASB16
ATAD3C
ATF7IP
ATP12A
ATP6V1C2
ATP8B3
ATP8B4
AVPR1B
B3GNT3
BAG3
BANK1
BCL9
BMP2
BMP3
BPIFB2
BRCA1
BTBD16
BTD
BTK
BTN1A1
BTN2A1
C14orf37
C16orf46
C19orf53
C1orf158
C1orf167
C1orf177
C1orf194
C1orf87
C1QTNF9B
C2orf16
C2orf61
C2orf70
C2orf73
C2orf74
C4B
C4orf33
C5orf45
C7
C7orf31
C7orf57
C7orf72
C9orf114
CABIN1
CACNA1S
CACNB2
CACNG6
CALCOCO2
CALR
CAPN5
CAPN8
CAPN9
CAPS2
CAPZA3
CASC1
CASC5
CBLC
CBS
CBWD3
CBW

1186 genes. Gene names listed above.

### 3.1.2) SIFT Scores
SIFT scores less than 0.05 are considered deleterious. Anything greater is considered tolerated. Lower SIFT scores are considered more deleterious.

## Question 7)
What is the lowest SIFT score of the deleterious variants?

## Answer 7)

In [17]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

10	122336645	A	G	0.000
10	125980182	C	T	0.000
10	128113592	C	G	0.000
10	26219214	C	A	0.000
10	46461688	A	C	0.000
10	46549695	C	G	0.000
10	46549695	C	T	0.000
10	48086	G	A	0.000
10	59792934	G	T	0.000
10	6224537	G	T	0.000


0.0 is the lowest SIFT score.

## Question 8)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 8)

In [18]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> Venter.SIFT4G.sift_score_0.txt
head -n10 Venter.SIFT4G.sift_score_0.txt

CHROM	POS	REF_ALLELE	ALT_ALLELE	GENE_NAME	REF_AMINO	ALT_AMINO	AMINO_POS	SIFT_SCORE	SIFT_PREDICTION
1	1956754	C	A	CFAP74	G	C	628	0.000	DELETERIOUS
1	3497541	C	T	MEGF6	G	R	1152	0.000	DELETERIOUS
1	11789390	A	G	C1orf167	R	G	810	0.000	DELETERIOUS
1	17334004	G	C	PADI4	G	A	112	0.000	DELETERIOUS
1	25321889	G	C	RHD	G	A	385	0.000	DELETERIOUS
1	54670856	T	C	MROH7-TTC4	V	A	534	0.000	DELETERIOUS
1	54801124	G	C	TTC22	L	V	14	0.000	DELETERIOUS
1	54801124	G	C	TTC22	L	V	14	0.000	DELETERIOUS
1	120889909	T	G	PPIAL4B	L	R	30	0.000	DELETERIOUS


## 3.2) Analysis of James Watson germline variants

In [19]:
%%bash
zcat Watson.vcf.gz > Watson.vcf
java -jar SIFT4G_Annotator_v2.4.jar -c -t -i Watson.vcf -d GRCh38.78  -r Watson.SIFT4G

Start Time for SIFT4G code: Mon Mar 27 01:43:46 PDT 2017
Updates:
No updates from server!! Please go to http:sift-dna.org for updates.

Started Running .......
Running in Multitranscripts mode

Chromosome	WithSIFT4GAnnotations	WithoutSIFT4GAnnotations	Progress
MT			0			1			Completed : 1/25
Y			119			20889			Completed : 2/25
22			3226			25060			Completed : 3/25
20			5293			69584			Completed : 4/25
13			10529			120358			Completed : 5/25
21			6616			52355			Completed : 6/25
X			4172			70126			Completed : 7/25
10			10105			166368			Completed : 8/25
9			9591			135019			Completed : 9/25
18			11786			83503			Completed : 10/25
19			9681			67255			Completed : 11/25
17			8476			82316			Completed : 12/25
7			13094			182383			Completed : 13/25
16			13877			101925			Completed : 14/25
14			13495			90997			Completed : 15/25
6			11816			193121			Completed : 16/25
11			15692			165100			Completed : 17/25
15			16943			82199			Completed : 18/25
12			17819			145339			Completed : 19/25
8			18021			156492			

## Question 9)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 9)
8476 annotated, 82316 unnannotated

## Question 10)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 10)

In [20]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1970


1970 deleterious variants.

## Question 11)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 11)

In [21]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> Watson.SIFT4G.genes_with_deleterious_variants.txt
wc -l Watson.SIFT4G.genes_with_deleterious_variants.txt
cat Watson.SIFT4G.genes_with_deleterious_variants.txt

1528 Watson.SIFT4G.genes_with_deleterious_variants.txt
A2ML1
AADACL3
AASDHPPT
ABCA5
ABCA9
ABCB5
ABCC10
ABCC11
ABCC8
ABCC9
ABCG8
ABHD17B
ABL1
ABO
AC073657.1
ACACB
ACAN
ACAT1
ACOT4
ACP5
ACSM4
ACSS1
ADA
ADAM15
ADAM21
ADAM30
ADAMTS16
ADAMTS17
ADAMTS18
ADAMTS7
ADAMTSL3
ADD2
ADH1A
ADH1C
ADH5
ADNP
ADRBK1
AGAP2
AGBL1
AGBL2
AGPAT3
AGPAT9
AHCTF1
AHNAK
AHNAK2
AK9
AKAP13
AKAP3
AKR1C2
ALDH1B1
ALDH1L1
ALDH3B2
ALG9
ALOX5AP
ALPK1
ALPK2
ALPK3
ALPP
ALX4
AMACR
AMICA1
AMIGO2
AMPD3
ANGEL1
ANK1
ANKK1
ANKLE1
ANKLE2
ANKRD12
ANKRD30A
ANKRD33
ANKRD35
ANKRD36
ANKRD36C
ANKRD60
ANKS1A
ANKS3
ANO1
ANO10
ANP32E
ANTXRL
ANXA13
AP4S1
APOA1BP
APOA5
APOB
APOBEC2
APOL1
APOL4
APPL1
AQP7
AQR
ARCN1
ARFGEF1
ARHGAP17
ARHGAP19
ARHGAP19-SLIT1
ARHGEF19
ARHGEF28
ARHGEF37
ARID2
ARID4B
ARMC9
ARPP21
ART1
ASAP2
ASB16
ASH1L
ASMTL
ASNA1
ASPG
ASPM
ASPSCR1
ATAD5
ATF7IP
ATG2B
ATG9B
ATP13A4
ATP6V0D1
ATP7B
ATP8B4
ATPAF2
ATXN1
AVEN
AVIL
AVPR1B
BAG3
BARD1
BCAS1
BCAS3
BCL11A
BCL2A1
BCLAF1
BEST4
BICD1
BIRC8
BMP2
BMP3
BMP4
BRCA1
BTC
BTK
BUB1B
C10o

1528 genes. Gene names listed above.

## Question 12)
What genes do Craig Venter and James Watson both have deleterious variants in? How many genes is this?

## Answer 12)

In [22]:
%%bash
join Venter.SIFT4G.genes_with_deleterious_variants.txt Watson.SIFT4G.genes_with_deleterious_variants.txt \
> Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt
cat Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt

524 Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt
A2ML1
ABCC8
AC073657.1
ACACB
ACAN
ADAMTSL3
ADH1C
AHNAK
AKAP13
AKR1C2
ALDH1B1
ALPK2
ALPK3
ALPP
ALX4
AMACR
AMPD3
ANKLE1
ANKRD30A
ANKRD33
ANKRD35
ANKRD36
ANKRD36C
ANKRD60
ANXA13
APOA1BP
APOB
APOL4
AQP7
ARHGEF19
ARHGEF37
ARMC9
ARPP21
ASB16
ATF7IP
ATP8B4
AVPR1B
BAG3
BMP2
BMP3
BRCA1
BTK
C14orf37
C1orf167
C1orf87
C2orf61
C2orf73
C4orf33
C7
C7orf31
C7orf57
C7orf72
C9orf114
CALR
CAPN5
CAPN8
CAPN9
CAPZA3
CASC5
CBWD7
CCDC178
CCDC18
CCDC181
CCDC40
CCDC6
CCDC64B
CCDC93
CD163
CD6
CDH11
CDHR2
CDK11A
CDK11B
CDK5RAP2
CENPQ
CEP120
CFAP69
CFAP74
CHIA
CHIT1
COASY
COL15A1
COL17A1
COL2A1
COL4A3
COL4A4
COL6A2
COL6A5
COL6A6
COMT
COQ7
COX10
CRYBG3
CRYGB
CTNNAL1
CTSE
CUBN
CWH43
CYFIP1
DAPL1
DDRGK1
DDX58
DDX60L
DHTKD1
DISC1
DLEC1
DLGAP2
DNAH1
DNAH14
DOCK6
DOCK8
DPYD
DPYSL2
DYX1C1
ECHDC3
ELAVL1
EMR2
EPPK1
EPS8L1
ERCC5
ERICH6B
ERP27
ESPL1
EVC2
FAM178A
FAM186A
FAM188B
FAM220A
FAM35A
FANCI
FANK1
FAT2
FBLIM1
FBN3
FBXW8
FHDC1
FLJ22184
FLVCR1
FPR1
FRAS1
F

Gene names provided above. 524 genes in common.

## Question 13)
What is the lowest SIFT score of the deleterious variants?

## Answer 13)

In [23]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

10	113766634	T	C	0.000
10	19387657	A	G	0.000
10	26157364	C	A	0.000
10	46461688	A	C	0.000
10	48086	G	A	0.000
10	59792934	G	T	0.000
10	62376867	C	T	0.000
10	86936837	C	G	0.000
10	89307233	A	T	0.000
10	95339252	C	A	0.000


0.0 is the lowest SIFT score.

## Question 14)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 14)

In [24]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> Watson.SIFT4G.sift_score_0.txt
head -n10 Watson.SIFT4G.sift_score_0.txt

CHROM	POS	REF_ALLELE	ALT_ALLELE	GENE_NAME	REF_AMINO	ALT_AMINO	AMINO_POS	SIFT_SCORE	SIFT_PREDICTION
1	1956754	C	A	CFAP74	G	C	628	0.000	DELETERIOUS
1	3497541	C	T	MEGF6	G	R	1152	0.000	DELETERIOUS
1	11789390	A	G	C1orf167	R	G	810	0.000	DELETERIOUS
1	12725782	C	T	AADACL3	P	L	280	0.000	DELETERIOUS
1	17334004	G	C	PADI4	G	A	112	0.000	DELETERIOUS
1	26367769	T	C	ZNF683	D	G	48	0.000	DELETERIOUS
1	26367769	T	C	ZNF683	D	G	48	0.000	DELETERIOUS
1	26367769	T	C	ZNF683	D	G	48	0.000	DELETERIOUS
1	28490968	C	T	PHACTR4	R	C	622	0.000	DELETERIOUS


## 3.3) Analysis of 1000 Genomes Sample Human Data

### 3.3.1) Calling variants from aligned sequencing data
The 1000 Genomes exome sequencing data for this sample is not yet in VCF format. We must use samtools mpileup and bcftools call to convert it.

For samtools mpileup, we use the following options:
 - '-u' generate uncompressed VCF/BCF output. This saves time on compression and decompression, since we pipe to bcftools.
 - '-g' generate output in BCF format. This is a more compact binary format, ideal for transferring between programs.
 - '-f' the FASTA file used as reference for the CRAM file. Required to determine if something varies from the reference, and to decompress the CRAM data.
 
For bcftools call, we use the following options to call variants:
 - '-f GQ,GP' output genotype quality and genotype probability. We care about GQ for filtering.
 - '-v' output variant sites only. We don't care about sites that match the reference.
 - '-m' we use the multiallelic caller, upon recommendation by the samtools website.
 - '-O v' output VCF formatted file.
 - '-o' output variants to the specified file

We connect the output of samtools mpileup to the input of bcftools using a pipe '|'.

In [25]:
%%bash
date
samtools-1.4/samtools mpileup \
-ugf GRCh38_full_analysis_set_plus_decoy_hla.fa \
NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram \
| bcftools-1.4/bcftools call \
-f GQ,GP \
-vmO v \
-o NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf
date

Sun Mar 26 01:52:24 PDT 2017
Sun Mar 26 04:28:51 PDT 2017


Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


### 3.3.2) Filtering variants by read depth, quality, and genotype quality
Not all variant calls are made equal. We want to avoid predicting the deleteriousness of variants that may not be real. So we use filtering to filter for the depth of sequencing at each variant coordinate, and the confidence the variant caller has in the variant. This is encapsulated in the DP, QUAL, and GQ fields.

The command bcftools filter is used to implement these filters.
 - '-i' specifies an expression for variants to include.
 - 'INFO/DP>10': We want raw read depth to be greater than 10
 - 'QUAL>20': We want the quality of any variant called here to be greater than 20
 - 'FMT/GQ>20': We want the genotype to be called with a confidence greater than 20.

We then combine these criteria using logical AND ('&&') to yield the final filter inclusion statement, '(QUAL>20)&&(INFO/DP>10)&&(FMT/GQ>20)'.

For more details on DP, QUAL, and GQ, see the guide from GATK (http://gatkforums.broadinstitute.org/gatk/discussion/1268/what-is-a-vcf-and-how-should-i-interpret-it).

In [26]:
%%bash
date
bcftools-1.4/bcftools filter -i '(QUAL>20)&&(INFO/DP>10)&&(FMT/GQ>20)' \
-O v \
-o NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf \
NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf
date

Sun Mar 26 04:28:51 PDT 2017
Sun Mar 26 04:28:56 PDT 2017


## Question 15)
How many variants are in the VCF before filtering? How many after filtering?

## Answer 15)

In [27]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf|grep -v '^#'|wc -l
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf|grep -v '^#'|wc -l

2254572
93617


2254572 variants before filtering.
93617 variants after filtering.

### 3.3.3) Annotating variants with SIFT4G

In [34]:
%%bash
java -jar SIFT4G_Annotator_v2.4.jar -c -t \
-i NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf \
-d GRCh38.78  \
-r NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G

Start Time for SIFT4G code: Mon Mar 27 11:51:12 PDT 2017
Updates:
No updates from server!! Please go to http:sift-dna.org for updates.

Started Running .......
Running in Multitranscripts mode

Chromosome	WithSIFT4GAnnotations	WithoutSIFT4GAnnotations	Progress
The following chromosomes (or scaffolds/contigs) are not found in the SIFT 4G database and will not be annotated:
HLA-B*08:33, Un_KN707967v1_decoy, HLA-DQB1*06:03:01, 19_KI270882v1_alt, Un_JTFH01000594v1_decoy, HLA-A*02:81, HLA-B*67:02, 5_KI270794v1_alt, Un_JTFH01000217v1_decoy, Un_JTFH01001433v1_decoy, Un_JTFH01001822v1_decoy, HLA-DRB1*15:02:01, Un_JTFH01001981v1_decoy, HLA-A*26:50, Un_KI270746v1, HLA-A*01:16N, HLA-A*03:02:01, 7_KI270808v1_alt, 8_KI270822v1_alt, HLA-B*55:12, HLA-DQA1*05:11, HLA-A*68:08:01, 10_KI270824v1_alt, 1_KI270764v1_alt, 19_KI270933v1_alt, Un_JTFH01001889v1_decoy, 19_GL949749v2_alt, Un_JTFH01000544v1_decoy, HLA-A*02:77, HLA-DQB1*05:03:01:01, HLA-DQB1*05:03:01:02, HLA-A*02:68, HLA-B*39:01:01:02L, HLA-B*67:01

## Question 16)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 16)
1571 annotated, 2169 unnannotated

## Question 17)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 17)

In [35]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1365


1365 deleterious variants.

## Question 18)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 18)

In [36]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
wc -l 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
cat 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt

1154 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
A2ML1
ABCA12
ABCA4
ABCA9
ABCC11
ABCC4
ABCD4
ABHD11
ABO
AC244230.1
ACACB
ACAN
ACSM5
ACSS1
ACTRT2
ADAM15
ADAM18
ADAMDEC1
ADAMTS18
ADAMTSL3
ADO
ADORA3
ADPGK
AFMID
AHNAK
AK2
AKAP1
AKAP13
AKAP2
AKAP3
AKR1C2
ALB
ALDH1B1
ALG6
ALPK2
ALPK3
ALPL
AMACR
AMICA1
AMPD3
ANAPC10
ANGEL1
ANKK1
ANKRD30A
ANLN
ANO10
ANO2
ANO3
ANXA7
AOAH
AP1G2
AP2S1
APH1B
APOA1BP
APOB
APOBEC3H
APOL1
APOL4
AQP12A
ARHGAP9
ARHGEF28
ARHGEF37
ARMC9
ARRDC4
ASB16
ASB18
ASH1L
ASMTL
ASPSCR1
ATF7IP
ATP10A
ATP6V1C2
ATRX
ATXN1
AVPR2
B3GNT3
B4GALNT2
BAG3
BANK1
BARD1
BCL9
BMP2
BMP3
BNIPL
BPIFB2
BRIP1
BST1
BTD
BTNL2
BTNL8
BUD13
C14orf37
C14orf80
C18orf54
C1orf112
C1orf177
C1orf87
C1QTNF6
C1QTNF9B
C2orf61
C2orf73
C3
C3orf20
C3orf30
C4orf33
C6orf15
C6orf222
C7
C7orf31
C7orf57
C7orf72
C9orf43
CACNA1H
CACNA1S
CAGE1
CALCA
CALCOCO2
CAPN11
CAPN12
CAPN9
CARD14
CASC1
CASC5
CAV2
CCAR1
CCDC116
CCDC122
CCDC13
CCDC141
CCDC157
CCDC18
CCDC180
CCDC181
CCDC88C
CCDC89
CCHCR1
CCNA1
CCND3
CCNH
CC

1154 genes. Gene names listed above.

## Question 19)
What genes do Craig Venter, James Watson, and this 1000 Genomes sample All have deleterious variants in? How many genes is this?

## Answer 19)

In [37]:
%%bash
join Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt \
1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt \
> Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
cat Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt

322 Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
A2ML1
ACACB
ACAN
ADAMTSL3
AHNAK
AKAP13
AKR1C2
ALDH1B1
ALPK2
ALPK3
AMACR
AMPD3
ANKRD30A
APOA1BP
APOB
APOL4
ARHGEF37
ARMC9
ASB16
ATF7IP
BAG3
BMP2
BMP3
C14orf37
C1orf87
C2orf61
C2orf73
C4orf33
C7
C7orf31
C7orf57
C7orf72
CAPN9
CASC5
CCDC18
CCDC181
CD163
CDH11
CDHR2
CDK11A
CDK11B
CDK5RAP2
CENPQ
CFAP69
CFAP74
CHIA
CHIT1
COASY
COL2A1
COL4A3
COL4A4
COL6A6
COQ7
COX10
CRYBG3
CRYGB
CUBN
CWH43
DAPL1
DDX58
DDX60L
DHTKD1
DISC1
DLEC1
DLGAP2
DOCK6
DOCK8
DPYSL2
ECHDC3
EPPK1
EPS8L1
ERCC5
ESPL1
EVC2
FAM188B
FAT2
FBLIM1
FBN3
FBXW8
FHDC1
FPR1
FRAS1
FREM2
FRG1B
FRMD4B
FUT2
FUT3
FUT9
GALNTL5
GALP
GBP3
GBP6
GCAT
GGT6
GIMAP6
GOLGA6L2
GORAB
GPR137C
GPR98
GPRIN2
GRIN3A
GSTA5
HEATR5A
HIBCH
HMGXB4
HPS4
HRNR
HSDL1
HUS1B
IFT88
IKBKAP
INMT
IQGAP3
ITGA11
ITGA9
ITGAE
ITGB4
ITPR2
KCNJ12
KIAA0753
KIAA1755
KLHDC1
KLRC3
KRT13
KRT32
KRTAP10-1
KRTAP10-11
KRTAP10-3
KRTAP10-4
KRTAP10-5
KRTAP12-3
KRTAP9-4
LARP1B
LCE3D
LMO7
LRP2
LYSMD4
MAP2K3
MA

Gene names provided above. 322 genes in common.

## Question 20)
What is the lowest SIFT score of the deleterious variants?

## Answer 20)

In [38]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

chr10	100506090	A	C	0.000
chr10	11755501	G	A	0.000
chr10	26219214	C	A	0.000
chr10	46549695	C	A	0.000
chr10	48086	G	A	0.000
chr10	73378933	C	T	0.000
chr10	97465888	G	A	0.000
chr1	11046609	T	C	0.000
chr11	108593482	T	C	0.000
chr11	26508237	C	T	0.000


0.0 is the lowest SIFT score.

## Question 21)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 21)

In [39]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls \
|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> 1KGenomesSample.SIFT4G.sift_score_0.txt
head -n10 1KGenomesSample.SIFT4G.sift_score_0.txt

CHROM	POS	REF_ALLELE	ALT_ALLELE	GENE_NAME	REF_AMINO	ALT_AMINO	AMINO_POS	SIFT_SCORE	SIFT_PREDICTION
chr1	1956754	C	A	CFAP74	G	C	628	0.000	DELETERIOUS
chr1	11046609	T	C	MASP2	D	G	120	0.000	DELETERIOUS
chr1	17334004	G	C	PADI4	G	A	112	0.000	DELETERIOUS
chr1	18483281	T	C	KLHDC7A	L	S	767	0.000	DELETERIOUS
chr1	25342976	T	G	TMEM50A	W	G	37	0.000	DELETERIOUS
chr1	26043403	G	T	SLC30A2	N	K	189	0.000	DELETERIOUS
chr1	26043403	G	T	SLC30A2	N	K	140	0.000	DELETERIOUS
chr1	54653861	C	T	MROH7	S	F	312	0.000	DELETERIOUS
chr1	54653861	C	T	MROH7	S	F	312	0.000	DELETERIOUS


# 4) References
1. Vaser R, Adusumalli S, Leng SN, Sikic M, Ng PC. SIFT missense predictions for genomes. Nat Protoc. 2016;11(1):1-9. (https://www.ncbi.nlm.nih.gov/pubmed/26633127)
2. Aken BL, Achuthan P, Akanni W, et al. Ensembl 2017. Nucleic Acids Res. 2017;45(D1):D635-D642. (https://www.ncbi.nlm.nih.gov/pubmed/27899575)
3. Auton A, Brooks LD, Durbin RM, et al. A global reference for human genetic variation. Nature. 2015;526(7571):68-74. (https://www.ncbi.nlm.nih.gov/pubmed/26432245)
4. Li H, Handsaker B, Wysoker A, et al. The Sequence Alignment/Map format and SAMtools. Bioinformatics. 2009;25(16):2078-9. (https://www.ncbi.nlm.nih.gov/pubmed/19505943)
5. Li H. A statistical framework for SNP calling, mutation discovery, association mapping and population genetical parameter estimation from sequencing data. Bioinformatics. 2011;27(21):2987-93. (https://www.ncbi.nlm.nih.gov/pubmed/21903627)
